In [1]:
import numpy as np
from numpy.random import choice

In [3]:
# check if the qbit is valid :Todo
q0 = np.array([0,1])
p1 = np.abs(q0)**2
print(int(round(p1.sum(),5)))

1


In [5]:
X = np.array([
    [0,1],
    [1,0]
])

q0 = np.dot(X,q0)
print("Final state:\t", q0)

Final state:	 [1 0]


In [6]:
psi = [1,0,0,0,0,0,0,0]
print("Initial state:", psi)

X = np.array([
    [0,1],
    [1,0]
])

I = np.identity(2)

O = np.kron(np.kron(I, I), X)

print("\nOperator:\n\n", O, "\n")

psi = np.dot(psi, O)
print("Final state:", psi)

Initial state: [1, 0, 0, 0, 0, 0, 0, 0]

Operator:

 [[0. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0.]] 

Final state: [0. 1. 0. 0. 0. 0. 0. 0.]


In [7]:
qbit = {
    0: 0,
    1: 0
}
for i in range(100):
    draw = choice(2,1,p=[0.1,0.9])
    if draw[0] == 0:
        qbit[0]+=1
    else:
        qbit[1]+=1
print(qbit)

{0: 7, 1: 93}


In [12]:
def get_ground_state(num_qubits):
    # return vector of size 2**num_qubits with all zeroes except first element which is 1
    if(num_qubits <=0):
        return "Please enter valid number greater than 0"
    q = [1,0]
    for i in range(num_qubits-1):
        q = np.kron(q,[1,0])
    return q

def get_operator(total_qubits, gate_unitary, target_qubits):
    # return unitary operator of size 2**n x 2**n for given gate and target qubits
    switcher_gate = {
        "x": x_gate,
        "y": y_gate,
        "z": z_gate,
        "h": hadamard_gate,
        "cx": cnot_gate
    }
    gate_func = switcher_gate.get(gate_unitary, lambda a,b: print("Invalid gate name"))
    return gate_func(total_qubits, target_qubits)

def set_qubit_gate(single_qubit_gate_operator, target_qubits, total_qubits):
    identity = np.identity(2)
    qubit_gates = {}
    for i in range(total_qubits):
        if i == target_qubits[0]:
            qubit_gates[i] = single_qubit_gate_operator
        else:
            qubit_gates[i] = identity
    return qubit_gates

def x_gate(total_qubits, target_qubits):            
    X_operator = np.array([
        [0,1],
        [1,0]
    ])
    qubit_gates = set_qubit_gate(X_operator, target_qubits, total_qubits)    
    operator = np.array([1])
    for j in range(total_qubits):
        operator = np.kron(operator,qubit_gates[j])            
    return operator

def y_gate(total_qubits, target_qubits):
    Y_operator = np.array([
        [0,0-1.j],
        [0+1.j,0]
    ])
    qubit_gates = set_qubit_gate(Y_operator, target_qubits, total_qubits)    
    operator = np.array([1])
    for j in range(total_qubits):
        operator = np.kron(operator,qubit_gates[j])            
    return operator  

def z_gate(total_qubits, target_qubits):
    Z_operator = np.array([
        [1,0],
        [0,-1]
    ])
    qubit_gates = set_qubit_gate(Z_operator, target_qubits, total_qubits)
    operator = np.array([1])
    for j in range(total_qubits):
        operator = np.kron(operator,qubit_gates[j])
    return operator

def hadamard_gate(total_qubits, target_qubits):
    h_operator = np.array([
        [1/np.sqrt(2),1/np.sqrt(2)],
        [1/np.sqrt(2),-1/np.sqrt(2)]
    ])
    qubit_gates = set_qubit_gate(h_operator, target_qubits, total_qubits)    
    operator = np.array([1])
    for j in range(total_qubits):
        operator = np.kron(operator,qubit_gates[j])    
    return operator

def cnot_gate(total_qubits, target_qubits):
    P0x0 = np.array ([
    [1,0],
    [0,0]
    ])
    
    P1x1 = np.array([
    [0, 0],
    [0, 1]
    ])
    
    X_operator = np.array([
        [0,1],
        [1,0]
    ])   
    identity = np.identity(2)
    
    operator1 = P0x0
    operator2 = P1x1
    
    if target_qubits[0]<target_qubits[1]:
        qubits = [*range(target_qubits[0],target_qubits[1]+1)]
        for i in range(target_qubits[1]-target_qubits[0]-1):
            operator1 = np.kron(operator1,identity)
            operator2 = np.kron(operator2,identity)
        operator = np.kron(operator1,identity) + np.kron(operator2,X_operator)
    else:
        qubits = [*range(target_qubits[1],target_qubits[0]+1)]
        for i in range(target_qubits[0]-target_qubits[1]-1):
            operator1 = np.kron(identity,operator1)
            operator2 = np.kron(identity,operator2)
        operator = np.kron(identity,operator1) + np.kron(X_operator,operator2)
        
    added_operator = False
    final_operator = [1]
    
    for j in range(total_qubits):
        if j not in qubits:
            final_operator = np.kron(final_operator,identity)
        elif j in qubits and added_operator==False:
            final_operator = np.kron(final_operator,operator)
            added_operator = True
    
    return final_operator
        
# print(get_operator(3,"cx",[1,2]))

def run_program(initial_state, program):
    total_qubits = int(np.log2(len(initial_state)))
    for i in program:
        matrix_operator = get_operator(total_qubits, i['gate'], i['target'])
        initial_state = np.dot(initial_state, matrix_operator)
    return initial_state

def get_counts(final_state, shots):
    prob = np.abs(final_state)**2
    length_state = len(prob)
    total_qubits = int(np.log2(length_state))
    qubit_result = {}
    qubits = []
    
    for i in range(length_state):
        qubit_result[np.binary_repr(i,width=total_qubits)] = 0
        qubits.append(np.binary_repr(i,width=total_qubits))
    for j in range(shots):
        result = choice(qubits,1,p=prob)
        qubit_result[result[0]]+= 1
#     print(qubits)
    return qubit_result
    
my_circuit = [
    { "gate": "x", "target": [0] },
    { "gate": "h", "target": [1] }
]

my_qpu = get_ground_state(2)

final_state = run_program(my_qpu, my_circuit)

print(final_state)

print(get_counts(final_state,1000))

[0.         0.         0.70710678 0.70710678]
{'00': 0, '01': 0, '10': 507, '11': 493}
